In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/kaggle/input/crowdedness-at-the-campus-gym/data.csv")

In [ ]:
print(df.columns)
print(df.info())

In [ ]:
df['date'] = pd.to_datetime(df['date'],utc=True)
df.info()
df['year'] = df['date'].dt.year
print(df.head())

In [ ]:
df.drop('timestamp',axis=1, inplace=True)
print(df.columns)

In [ ]:
df.drop('date',axis=1, inplace=True)

In [ ]:
X = df.drop('number_people',axis=1)
y = df['number_people']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25 ,random_state=15)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [ ]:
def calculate_model_metrics(true, predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)
    return mae,rmse,r2_square

In [ ]:
models = {
    "Linear Regession": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Rnadom Forest Regressor": RandomForestRegressor(),
}

In [ ]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred =model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae,model_train_rmse,model_train_r2 = calculate_model_metrics(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2 = calculate_model_metrics(y_test,y_test_pred)

    print(list(models.values())[i])
    print("Evaluation for Training Set")
    print("Rmse:",model_train_rmse)
    print("MAE:", model_train_mae)
    print("R_2:", model_train_r2)

    print("--------------------------------------")

    print("Evaluation for Test Set")
    print("Rmse:", model_test_rmse)
    print("MAE:", model_test_mae)
    print("R_2:", model_test_r2)

    print("--------------------------------------")
    print("\n")


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_test_pred)
plt.xlabel("Gerçek Kişi Sayısı (Test Seti)")
plt.ylabel("Tahmin Edilen Kişi Sayısı (Model: Random Forest)")
plt.title("Gerçek ve Tahmin Edilen Yoğunluk Karşılaştırması (RF)")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', lw=2, linestyle='--')
plt.show()